# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [82]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from pandas_datareader import wb

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2


# user written modules
from plot_function import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction

In this project we would like to analyse the impacts of net migration on 4 different variables: GDP, employment rate, labour force and wage. We took on this project because migration is a big debate in the majority of rich countries, and especially with the rise of the far-right in Europe.

We chose to study the USA, since it is a net imigration country, and Romania, since it is a net emigration country. We decided to use the data from 1991 - the first year we had all information - until 2019, to not take into account the pandemic. 

# Read and clean data

Import your data, either through an API or manually, and load it. 

In [83]:
#setup period
start_year = 1991
end_year = 2019

#download net migration data from the World Bank
wb_migration = wb.download(indicator='SM.POP.NETM', country=[ 'USA', 'ROU'], start=start_year, end=end_year)

#rename the column
wb_migration = wb_migration.rename(columns = {'SM.POP.NETM':'net migration'})

#reset data
wb_migration = wb_migration.reset_index()

#convert year from an object to an integral type
wb_migration.year = wb_migration.year.astype(int) 

#convert country from an object to a string type
wb_migration.country = wb_migration.country.astype('string') 


In [131]:
#load data
wb_migration.to_csv('migration.csv', index=False)

In [85]:
#download labour force data from the World Bank
wb_labour = wb.download(indicator='SL.TLF.TOTL.IN', country=[ 'USA', 'ROU'], start=start_year, end=end_year)

#rename the column
wb_labour = wb_labour.rename(columns = {'SL.TLF.TOTL.IN':'labor_force'})

#reset data
wb_labour = wb_labour.reset_index()

#convert year from an object to an integral type
wb_labour.year = wb_labour.year.astype(int) 

#convert country from an object to a string type
wb_labour.country = wb_labour.country.astype('string') 

In [86]:
#load data
wb_labour.to_csv('labour_force.csv', index=False)

In [87]:
#download labour force data from the World Bank
wb_gdp = wb.download(indicator='NY.GDP.MKTP.CD', country=[ 'USA', 'ROU'], start=start_year, end=end_year)

#rename the column
wb_gdp = wb_gdp.rename(columns = {'NY.GDP.MKTP.CD':'gdp'})

#reset data
wb_gdp = wb_gdp.reset_index()

#convert year from an object to an integral type
wb_gdp.year = wb_gdp.year.astype(int) 

#convert country from an object to a string type
wb_gdp.country = wb_gdp.country.astype('string') 


In [88]:
#load data
wb_gdp.to_csv('gdp.csv', index=False)

In [89]:
#download wage data from the World Bank
wb_wage = wb.download(indicator='SL.EMP.WORK.ZS', country=[ 'USA', 'ROU'], start=start_year, end=end_year)

#rename the column
wb_wage = wb_wage.rename(columns = {'SL.EMP.WORK.ZS':'wage'})

#reset index
wb_wage = wb_wage.reset_index()

#convert year from an object to an integral type
wb_wage.year = wb_wage.year.astype(int) 

#convert country from an object to a string type
wb_wage.country = wb_wage.country.astype('string') 

In [90]:
#load data
wb_wage.to_csv('wage.csv', index=False)

In [91]:
#download employment data from the World Bank
wb_employ = wb.download(indicator='SL.EMP.TOTL.SP.ZS', country=[ 'USA', 'ROU'], start=start_year, end=end_year)

#rename the column
wb_employ = wb_employ.rename(columns = {'SL.EMP.TOTL.SP.ZS':'employment percentage'})

#reset index
wb_employ = wb_employ.reset_index()

#convert year from an object to an integral type
wb_employ.year = wb_employ.year.astype(int) 

#convert country from an object to a string type
wb_employ.country = wb_employ.country.astype('string') 

In [92]:
#load data
wb_employ.to_csv('employment_ratio.csv', index=False)

In [93]:
#merge the two data
var_list = [wb_gdp, wb_employ, wb_labour, wb_wage]

wb = wb_migration

for i in var_list:
    wb = pd.merge(wb, i, how = 'outer', on = ['country','year']);

wb.sample(10)

,country,year,net migration,gdp,employment percentage,labor_force,wage
48,United States,2000,1479676,1.025095e+13,63.297001,146165420,92.440002
10,Romania,2009,-129392,1.741037e+11,50.705002,9338466,67.220001
42,United States,2006,1113259,1.381559e+13,61.921001,154153998,92.599998
43,United States,2005,1129462,1.303920e+13,61.515999,152044687,92.529999
21,Romania,1998,-89939,4.169412e+10,61.435001,11808407,57.939999
46,United States,2002,1168499,1.092911e+13,61.548000,148166545,92.639999
3,Romania,2016,-64002,1.852869e+11,50.550999,8915255,73.389999
47,United States,2001,1335725,1.058193e+13,62.519001,147200478,92.519997
1,Romania,2018,-57865,2.433171e+11,52.675999,8999055,74.779999
31,United States,2017,1377630,1.947734e+13,59.590000,163971525,93.750000


## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

In [94]:
#creation of a plot with two y-axis
def plot_func(country, data):
    country_data = wb[wb['country'] == country]
    fig, ax1 = plt.subplots()

    color = 'tab:blue'
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Net migration', color=color)
    ax1.plot(country_data['year'], country_data['net migration'], color=color)
    

    ax2 = ax1.twinx()  

    color = 'tab:purple'
    ax2.set_ylabel(str(data), color=color)  
    ax2.plot(country_data['year'], country_data[data], color=color)


    plt.title(f'{country} Net Migration and gdp over Time')
    fig.tight_layout()  
    plt.show()

#create of two dropdown
country_widget = widgets.Dropdown(options=wb['country'].unique(), description='Country:')
data_list=wb.columns.tolist()[3:]
data_widget = widgets.Dropdown(options=data_list, description='Data:')

#create an interactive plot
widgets.interact(plot_func, country=country_widget, data=data_widget);


interactive(children=(Dropdown(description='Country:', options=('Romania', 'United States'), value='Romania'),…

In [95]:
#creation of a scatter plot
def scatter_func(country, data):
    country_data = wb[wb['country'] == country]
    fig, ax1 = plt.subplots()

    color = 'tab:blue'
    ax1.set_xlabel('Net migration')
    ax1.set_ylabel(str(data), color=color)
    ax1.scatter(country_data['net migration'], country_data[data], color=color)
    
    slope, intercept = np.polyfit(country_data['net migration'],country_data[data],1)
    ax1.plot(country_data['net migration'],country_data['net migration']*slope+intercept)

    plt.title(f'{country} Net Migration and data over Time')
    fig.tight_layout()  
    plt.show()

#create of two dropdown
country_widget = widgets.Dropdown(options=wb['country'].unique(), description='Country:')
data_list=wb.columns.tolist()[3:]
data_widget = widgets.Dropdown(options=data_list, description='Data:')

#create an interactive plot
widgets.interact(scatter_func, country=country_widget, data=data_widget);

interactive(children=(Dropdown(description='Country:', options=('Romania', 'United States'), value='Romania'),…

**Interactive plot** :

Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

In [130]:
wb_US = wb[wb["country"] == "United States"]
wb_Rom = wb[wb["country"] == "Romania"]

var_list = ["gdp", "employment percentage", "labor_force", "wage"]

dict_var = {"country":["United States", "United States", "Romania", "Romania"], "measure":["corr","R^2 (%)","corr","R^2 (%)"]}

for data in var_list:
    US_data = wb_US["net migration"].corr(wb_US[data])
    Rom_data = wb_Rom["net migration"].corr(wb_Rom[data])

    dict_var[data] = [f"{US_data:.3f}" , f"{100 * US_data**2:.0f}", f"{Rom_data:.3f}", f"{100 * Rom_data**2:.0f}"]


corr_table = pd.DataFrame(dict_var).set_index(["country","measure"])
corr_table

gdp employment percentage labor_force    wage
country       measure                                                  
United States corr     -0.670                 0.376      -0.748  -0.689
              R^2 (%)      45                    14          56      48
Romania       corr      0.251                 0.060      -0.105   0.378
              R^2 (%)       6                     0           1      14

As we can see, our correlation results about GDP are inconclusive, since the US has a negative correlation, meaning that an increase in net migration is normally accompanied by a decrease in GDP, while Romania has a small, but positive correlation, showing the opposite effect. The coefficient of determination of Romania is really small, meaning the variation of the net migration rate can only explain 6% of the variation in the GDP. On the other hand, the US's net migration rate can explain 45% of the GDP variation.

The employment percentage between both countries have a posisitive correlation, although very small in both, meaning an increase of the net migration rate, the employment rate increases too. But, this correlation only explains 14% of the variation in the US and 0% of the variation in Romania, therefore not being correlated.

Labor force in both countries has a negative correlation, meaning a bigger net migration rate causes a negative influence in the labor force in a country. In the US, the variation in the net migration rate explains 56% of the variation in the labor force, while in Romania, it only explains 1%.

Finally, the wage in both countries has a inconclusive correlation. The United States show a negative correlation, meaning a bigger net migration rate decreases wage, while the opposite happens in Romania. In the US, the wage variation can only be explained by 48% of the variation in the net migration rate, while in Romania, it only explains 14%.

# Conclusion

We can conlude that net migration rate does not explain much of GDP, employment, labor force and wage in the US and in Romania. Therefore, it is not a good way of predicting all of those variables, and other variables might be more closely related to them.

The labor force in the US has the best correlation with net migration, of 56%, but even then we can't sustain the claim that migration is bad, since correlation is not causation.